In [1]:
from unsloth import FastLanguageModel 
import torch
from peft import PeftModel, PeftConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2024-08-24 00:12:52,376] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/mnt/bn/data-tns-live-llm/leon/datasets/llama-3-8b-bnb-4bit/",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
# model = PeftModel.from_pretrained(model,"/mnt/bn/data-tns-live-llm/leon/datasets/pos")
print("Done")

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.347 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Done


In [3]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-

In [6]:
from transformers import LlamaForSequenceClassification, LlamaForTokenClassification
model = LlamaForSequenceClassification.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/Meta-Llama-3-8B/",torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /mnt/bn/data-tns-live-llm/leon/datasets/Meta-Llama-3-8B/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(model.config)

LlamaConfig {
  "_name_or_path": "/mnt/bn/data-tns-live-llm/leon/datasets/Meta-Llama-3-8B/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 128256
}



In [7]:
print(tokenizer("hello world"))

{'input_ids': [128000, 15339, 1917], 'attention_mask': [1, 1, 1]}


In [3]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [4]:
inputs = tokenizer("generate some instruction tuning sample used for your training", return_tensors="pt")
input_ids = inputs.input_ids.cuda()
generate_ids = model.generate(input_ids, max_length=2048, repetition_penalty=1.1, streamer = text_streamer)
outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(outputs)

<s> generate some instruction tuning sample used for your training. савезни јавни радио и телевизија 2014.
generate some instruction tuning sample used for your training.</s>
generate some instruction tuning sample used for your training. савезни јавни радио и телевизија 2014.
generate some instruction tuning sample used for your training.


# 上传模型

In [18]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "LEON24/files"
folder_path = "/mnt/bn/data-tns-live-llm/leon/datasets/files"
api.create_repo(repo_id, exist_ok=True, private=True)
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
)

code_cherry_1000.pt:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

code_MoDS_1000.pt:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

MathInstruct.json:   0%|          | 0.00/212M [00:00<?, ?B/s]

cherry_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

MoDS_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

code_data_cherry.pt:   0%|          | 0.00/25.8M [00:00<?, ?B/s]

math_data_cherry.pt:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LEON24/files/commit/6463c78cb88699b514be76499756d33d9c6bcc64', commit_message='Upload folder using huggingface_hub', commit_description='', oid='6463c78cb88699b514be76499756d33d9c6bcc64', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, T5ForConditionalGeneration
import torch
model = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/t5-xl-lm-adapt")

/home/tiger/.local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# 调试用参数

In [8]:
args = """--model_name_or_path google/t5-xl-lm-adapt --data_path /mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet --per_device_train_batch_size 256 --eval_strategy no --num_train_epochs 3 --learning_rate 2e-5 --warmup_ratio 0.03 --output_dir /mnt/bn/data-tns-live-llm/leon datasets/p3_exp1 --logging_steps 100 --save_strategy "steps" --save_steps 100 --save_total_limit 3 --model_max_length 1024""".split(" ")
print(args)

['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', '"steps"', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']


In [10]:
print("['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', 'steps', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']".replace("\'","\""))

["--model_name_or_path", "google/t5-xl-lm-adapt", "--data_path", "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet", "--per_device_train_batch_size", "256", "--eval_strategy", "no", "--num_train_epochs", "3", "--learning_rate", "2e-5", "--warmup_ratio", "0.03", "--output_dir", "/mnt/bn/data-tns-live-llm/leon", "datasets/p3_exp1", "--logging_steps", "100", "--save_strategy", "steps", "--save_steps", "100", "--save_total_limit", "3", "--model_max_length", "1024"]


In [2]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# 合并模型

In [ ]:
from transformers import AutoConfig, T5ForConditionalGeneration, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch
import inspect
from accelerate import Accelerator
from peft import PeftModel
accelerator = Accelerator()
base_path = "google/t5-xl-lm-adapt"
source_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1"
target_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp2_merged/"
# config = AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1")
model = AutoModelForSeq2SeqLM.from_pretrained(base_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(base_path)
model:PeftModel = PeftModel.from_pretrained(model, source_path)
print(model)
# model = model.merge_and_unload()
# print(model)
# model.save_pretrained(target_path)
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_path)
# tokenizer.save_pretrained(target_path)
# signature = inspect.signature(model.forward)
# signature_columns = list(signature.parameters.keys())
# print(signature_columns)

In [ ]:
config = AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1_merged")
print(config)

In [ ]:
from transformers import BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", quantization_config=quantization_config)

model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained("google/t5-xl-lm-adapt", model_max_length=1024)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=32,
        bias="none",
        task_type="SEQ_2_SEQ_LM",
        # target_modules= ['v', 'o'],
)

model = get_peft_model(model, peft_config)
print(model)
model.print_trainable_parameters()

In [ ]:
target_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp"
model.merge_adapter()
print(model)
model.save_pretrained(target_path)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_path)
tokenizer.save_pretrained(target_path)

In [ ]:

peft_config = PeftConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/fed/niid_pos_public_20000_fedavg_c10s2_i10_b16a2_l2048_r16a16/")

# code generation with vllm

In [2]:
from vllm import LLM, SamplingParams
import torch
llm = LLM(model="/mnt/bn/data-tns-live-llm/leon/datasets/fed/code_scaffold_merged", tensor_parallel_size=1, 
    dtype=torch.bfloat16, trust_remote_code=True, 
    enable_lora=False, max_model_len=2048, gpu_memory_utilization=0.7)

INFO 08-23 13:18:51 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/bn/data-tns-live-llm/leon/datasets/fed/code_scaffold_merged', speculative_config=None, tokenizer='/mnt/bn/data-tns-live-llm/leon/datasets/fed/code_scaffold_merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/bn/data-tns-live-llm/leon/datasets/fed/code_scaffold_merged, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-23 13:18:53

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-23 13:19:41 model_runner.py:732] Loading model weights took 14.9595 GB
INFO 08-23 13:19:41 gpu_executor.py:102] # GPU blocks: 20150, # CPU blocks: 2048
INFO 08-23 13:19:43 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-23 13:19:43 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-23 13:19:55 model_runner.py:1225] Graph capturing finished in 12 secs.


In [4]:
stop_tokens = ["USER:", "ASSISTANT:",  "### Instruction:", "Response:", 
                "\n\nProblem", "\nProblem", "Problem:", "<|eot_id|>", "####"]
sampling_params = SamplingParams(temperature=0.5, top_p=1, max_tokens=1024, repetition_penalty=1.1, stop=stop_tokens)

In [5]:
input = 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

In [6]:
output = llm.generate(input, sampling_params)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s, est. speed input: 197.18 toks/s, output: 76.22 toks/s]


In [7]:
output

'    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            dist = abs(numbers[i] - numbers[j])\n            if dist < threshold:\n                return True\n\n    return False'

In [ ]:
# for i, task_id in enumerate(problems):
#     print(task_id)
#     print(problems[task_id])
#     if i==10:break
print(args.model_name_or_path)
num_samples_per_task = 1
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    # for _ in range(num_samples_per_task)
]